<h3>Design a Search Engine and Implement PageRank</h3>

Keyword-based search engines maintain a collection of several billion documents. One of the key computations performed by a search engine is to retrieve all the documents that contain the keywords contained in a given query. This is a nontrivial task in part because it must be performed in a few tens of milliseconds.

Let's consider a smaller version of the problem where the collection of documents can fit into a computer's RAM. 

<b>The Problem</b>

Given a million documents with an average size of 10KB, design a program that can efficiently return the subset of documents containing a given set of words. (<i>Hint: think of a book's index</i>)

<b>The Solution</b>

The predominant way of doing this is to build inverted indices. For each word, we store a sequence of locations where the word occurs. The sequence itself could be represented as an array, or as a linked list. Location is defined to be a document ID and the offset in the document. The sequence is stored in a sorted order of locations (first ordered by document ID, then by offset). When we are looking for documents that contain a set of words, what we need to do is find the intersection of sequences for each word. Since the sequences are already sorted, the intersection can be done in time proportional to the aggregate length of the sequences. Some optimizations to consider are listed below.

* <i>Compression</i>: compressing the inverted index helps with both the ability to index more documents, as wel as memory locality (increasing the cache hit rate). Since we are storing sorted sequences, one way of compressing is to use delta compression where we only store the difference between successive entries. The deltas can be represented in fewer bits.
* <i>Caching</i>: the distribution of queries is usually fairly skewed and it helps a great deal to cache the results of some of the most frequent queries.
* <i>Frequency-based optimization</i>: search results often do not need to return every document; only the top ten or so will do. These high quality documents will answer most of the queries. Therefore, we can make two inverted indicies, one with the high quality documents that stays in RAM, and one with the remaining documents that stays on disk. This way if we can keep the number of queries that require the secondary index to a small enough number, then we can still maintain a reasonable throughput and latency.
* <i>Intersection order</i>: since the total intersection time depends on the total size of sequences, it makes sense to intersect the words with smaller sets first. For example, if we are looking for "USA GDP 2009", it would make sense to intersect the lists for GDP and 2009 before trying to intersect the sequence for USA (which will be a much larger set).

Multi-level indexing (similar to n-grams in NLP) would improve accuracy in documents. For a high-priority web-page, we can decompose the page into paragraphs and sentences, which are indexed individually. That way the intersections for the words might be within the same context. We can pick results with closer index values from these sequences.

Below is some code from 2 relevant problems: (13.1 Sorted Array Intersection, and 12.6 Find Smallest SubArray Covering all Values)

<h3>PageRank</h3>

The internet can be viewed as a large graph, where web pages themselves represent nodes, and their links (direct connections to other web pages) can be seen as the edges of the graph. These links (edges) are directed. The underlying idea for PageRank is the following: <i>a page is important, if other important pages link to it.</i> Each link is viewed as a vote -- a de facto recommendation for the importance of that page. So, the PageRank algorithm is simply a counter that tallies up each of the web page's votes for other web pages. However, not all votes are equally important; a vote from a page with low importance (a page with few inlinks) should be adjusted to count less than a vote from an important page (with many inlinks). Each vote's importance is divided by the number of different votes a page casts, i.e., with a single outlink, all the weight is put towards the sole linked page, but if 100 outlinks are present, they all get $1/100th$ of the total weight.

The PageRank algorithm assigns a rank to a web page based on the number of "important" pages that link to it. Here is how to implement it:

(1.) Build a Matrix $A$ based on the hyperlink structure of the web. Specifically, $A_{ij} = \frac{1}{d_{j}}$ if page $j$ links to page $i$; here $d_{j}$ is the number of distinct pages linked from page $j$.

(2.) Find $X$ satisfying $X = \epsilon\textbf[1] + (1 - \epsilon)AX.$

<h3>Links</h3>

* EPI Textbook
* https://www2.it.uu.se/edu/course/homepage/projektTDB/vt04/projekt5/website/report.pdf